# Dataset preprocessing

In [1]:
import pandas as pd 
import urllib

In [5]:
# Import dataset
dataset = pd.read_csv("selfpost/rspct.tsv", sep="\t")
dataset["text"] = dataset["title"] + " . " + dataset["selftext"]
dataset.loc[:3, "text"]

0    Remember your command line switches... . Hi there,  <lb>The usual. Long time lerker, first time poster, be kind etc. Sorry if this isn't the right place...<lb><lb>Alright. Here's the story. I'm an independent developer who produces my own software. We're going to call me well, $me.<lb><lb>I work with $dev who helps to produce software with me. We use $PopularVersionControl.<lb><lb>We're trying to remove a branch that was created by mistake.  The branch is beta1. We want just beta.<lb><lb>&gt; $me: "$dev, can you rename that branch because we're going to use just two. I don't want to keep up with 80 quintilian branches."  <lb>&gt; $dev: "sure, one second."<lb><lb>Five minutes later...<lb><lb>&gt; $dev: "[CurseWords] I want beta1 to die!"  <lb>&gt; $me: "What happened?"<lb><lb>Lots of removed dialog where $dev explains what he did...<lb><lb>&gt; $me: "Did you try $PopularVersionControl with -u?"  <lb>&gt; $dev: "[Cursing] That would be why!"<lb><lb>In short. Always check your comman

In [ ]:
# Load another file for bpe and concat it with dataset data.
import requests
corpus_url = "https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt"
r = requests.get(corpus_url) # create HTTP response object
with open("botchan.txt",'wb') as f:
    f.write(r.content)
with open("botchan.txt", "a", encoding="utf-8") as file: 
    for row in dataset["text"]:
        file.write(row + ". ")


In [15]:
import spacy
import sentencepiece as spm
from tqdm import tqdm
from functools import lru_cache


def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 


def lemmatize_words(text): 
    # Text input is string, returns lowercased strings.
    return [token.lemma_ for token in nlp(text)]

def normalize_text(text):
    return " ".join([word for word in lemmatize_words(sp.decode_ids(sp.tokenize(text)))])

In [16]:
tqdm.pandas()
spm.SentencePieceTrainer.train('--input=botchan.txt --model_prefix=model --vocab_size=5239 --bos_id=-1 --pad_id=1')

C:\Users\TOPAPEC\anaconda3\envs\coursework\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [27]:
from multiprocessing import Pool
from joblib import Parallel, delayed
import pickle
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

sp = spm.SentencePieceProcessor()

sp.load('model.model')

# Я не уверен, насколько это хорошая практика, но я использовал sentencepiece, чтобы очистить текст от кучи склеенных слов, 
# а потом токенизировал nltk токенайзером (мне не очень понятно, как бить при помощи sentencepiece сразу на слова, а не члены его словаря)
with open(f"tmp\\dataset_dumps\\preprocess_dumps\\dataset_normalized\\tokenized.pkl", "rb") as file:
    dataset = pickle.load(file) 

# dataset["text"] = dataset["text"].progress_apply(lambda x : sp.decode_ids(sp.tokenize(x)))

# with open("tmp\\dataset_dumps\\preprocess_dumps\\dataset_normalized\\tokenized.pkl", "wb") as file: 
#     pickle.dump(dataset, file)

In [47]:
dataset.loc[:5, :]

,id,subreddit,title,selftext,text
0,6d8knd,talesfromtechsupport,Remember your command line switches...,"Hi there, <lb>The usual. Long time lerker, fi...",Remember your command line switch ... . Hi the...
1,58mbft,teenmom,"So what was Matt ""addicted"" to?",Did he ever say what his addiction was or is h...,So what wa Matt `` addicted '' to ? . Did he e...
2,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,No Club Colors . Funny story . I went to colle...
3,6ti6re,ringdoorbell,"Not door bell, but floodlight mount height.",I know this is a sub for the 'Ring Doorbell' b...,"Not door bell , but floodlight mount height . ..."
4,77sxto,intel,Worried about my 8700k small fft/data stress r...,"Prime95 (regardless of version) and OCCT both,...",Worried about my 8 ⁇ 00k small fft ⁇ data stre...
5,5qw3x0,residentevil,What if Saddler won?,I just wanted to start a thread about what wou...,What if Saddler won ? . I just wanted to start...


In [1]:
# nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])
# with open("tmp\\dataset_dumps\\preprocess_dumps\\dataset_normalized\\lemmatized_after_tokenization.pkl", "rb") as file:
#     istart, dataset = pickle.load(file)
# for i, item in enumerate(tqdm(nlp.pipe(dataset["text"][istart:], n_process=8), total=dataset.shape[0])):
#     dataset.loc[i, "text"] = item
#     if i % 5000 == 4999:
#         with open("tmp\\dataset_dumps\\preprocess_dumps\\dataset_normalized\\lemmatized_after_tokenization.pkl", "wb") as file: 
#             pickle.dump((i, dataset), file)

# Todo pos_tag
import pandas as pd
import pickle
import nltk
from nltk import WordNetLemmatizer

# wnl = WordNetLemmatizer()
# def lemmatize(text):
#     return " ".join([wnl.lemmatize(word) for word in nltk.word_tokenize(text)])

# dataset["text"] = dataset["text"].progress_apply(lemmatize)

# with open("tmp\\dataset_dumps\\preprocess_dumps\\dataset_normalized\\lemmatized_after_tokenization.pkl", "wb") as file:
#     pickle.dump(dataset, file)

with open("tmp\\dataset_dumps\\preprocess_dumps\\dataset_normalized\\lemmatized_after_tokenization.pkl", "rb") as file:
    dataset = pickle.load(file)

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
print(dataset.loc[:3, "text"])

0    Remember your command line switch ... . Hi there , ⁇ lb ⁇ The usual . Long time lerker , first time poster , be kind etc . Sorry if this is n't the right place ... ⁇ lb ⁇ lb ⁇ Alright . Here 's the story . I 'm an independent developer who produce my own software . We 're going to call me well , ⁇ me . ⁇ lb ⁇ lb ⁇ I work with ⁇ dev who help to produce software with me . We use ⁇ Popular ⁇ ersionControl . ⁇ lb ⁇ lb ⁇ We 're trying to remove a branch that wa created by mistake . The branch is beta1 . We want just beta . ⁇ lb ⁇ lb ⁇ gt ; ⁇ me : `` ⁇ dev , can you rename that branch because we 're going to use just two . I do n't want to keep up with 80 quintilian branch . '' ⁇ lb ⁇ gt ; ⁇ dev : `` sure , one second . '' ⁇ lb ⁇ lb ⁇ Five minute later ... ⁇ lb ⁇ lb ⁇ gt ; ⁇ dev : `` [ CurseWords ] I want beta1 to die ! '' ⁇ lb ⁇ gt ; ⁇ me : `` What happened ? '' ⁇ lb ⁇ lb ⁇ Lots of removed dialog where ⁇ dev explains what he did ... ⁇ lb ⁇ lb ⁇ gt ; ⁇ me : `` Did you try ⁇ Popular ⁇ er

<ipython-input-4-30f7f08f06f3>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [11]:
import nltk
import numpy as np


def encode(sentence, num_tokens : int, encoder_dict : dict, embeddings_dict : dict) -> np.ndarray:
    tokenized = nltk.word_tokenize(sentence)
    for word in tokenized:
        word_lower = word.lower()
        if word_lower not in encoder_dict and word_lower in embeddings_dict:
            encoder_dict[word_lower] = len(encoder_dict)
    array = np.ones((num_tokens))
    for i, word in enumerate(tokenized):
        if i >= num_tokens:
            break
        array[i] = encoder_dict.get(word.lower(), 0)
    return array

def decode(encoded_sentence, decoder_dict):
    return " ".join([decoder_dict[idx] for idx in encoded_sentence])

In [12]:
with open("tmp\\dataset_dumps\\preprocess_dumps\\embeddings\\glove_reddit_dict.pkl", "rb") as file:
    embeddings_dict = pickle.load(file)

In [17]:
import functools

from sklearn.preprocessing import LabelEncoder


# num_tokens = 256
# encoder_dict = {}
# decoder_dict = {}
# encoder_dict["⁇"] = 0
# encoder_dict["<pad>"] = 1
# X_encoded = np.array(dataset["text"].progress_apply(functools.partial(encode, num_tokens=num_tokens, encoder_dict=encoder_dict, embeddings_dict=embeddings_dict)))
# # for i in tqdm(range(dataset.shape[0])):
# #     dataset["text"][i] = encode(dataset["text"][i], num_tokens, encoder_dict)
# le = LabelEncoder()
# y = le.fit_transform(dataset["subreddit"])
# y_dict = le.get_params()
# X_encoded = np.stack(X_encoded).astype(np.int64)

# with open("tmp\\dataset_dumps\\preprocess_dumps\\dataset_normalized\\X_y_encoderDict_le_preprocessed.pkl", "wb") as file:
#     pickle.dump((X_encoded,y,encoder_dict,le), file)

100%|██████████████████████████████████████████████████████████████████████| 1013000/1013000 [14:22<00:00, 1173.90it/s]


In [18]:
X_encoded[3]

array([202, 184, 226,  10, 227, 228, 229, 230,   7,   7,  33, 133,  24,
        25,  58, 231, 208,  12,   0, 232, 187, 227, 233, 132, 234,  12,
       228,  91,  33, 235, 236,  12, 237, 138, 238, 239, 208,  12, 228,
       166,  12, 211,  93,  40, 240,  10, 227,  12, 241,  25,  60, 150,
        31, 124, 242, 243, 244, 245, 246,   0, 247, 248, 244,  12, 249,
        10, 250, 248, 251, 252,  12, 249, 253,  46,  12, 254,   0,   0,
        11,   0,  11,   0,  25,  60, 255, 256,  46, 229,  91,  12, 257,
       115,   0, 248,  25, 258,   7, 132, 218, 259, 241, 260, 150, 261,
       262,  60,  91,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   

In [45]:
def scipy_sparse_to_pytorch(matrix):
    values = matrix.data
    indices = np.vstack((matrix.tocoo().row, matrix.tocoo().col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = matrix.shape

    return torch.sparse_coo_tensor(i, v, torch.Size(shape), dtype=torch.int64)

In [2]:
import pickle
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

random_seed = 42
with open("tmp\\dataset_dumps\\preprocess_dumps\\dataset_normalized\\X_y_encoderDict_le_preprocessed.pkl", "rb") as file:
    X, y, encoder_dict, le = pickle.load(file)
# ohe = OneHotEncoder()
# y = ohe.fit_transform(y.reshape(-1,1)).astype(np.int64)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=random_seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=random_seed)
X_train, X_val, X_test = tuple([torch.tensor(array.astype(np.int64)) for array in [X_train, X_val, X_test]])
# y_train, y_val, y_test = tuple([scipy_sparse_to_pytorch(array) for array in [y_train, y_val, y_test]])
y_train, y_val, y_test = tuple([torch.tensor(array.astype(np.int64)) for array in [y_train, y_val, y_test]])

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

(torch.Size([162080, 256]),
 torch.Size([40520, 256]),
 torch.Size([810400, 256]),
 torch.Size([162080]),
 torch.Size([40520]),
 torch.Size([810400]))

# Dataloader

In [3]:
import torch
import random
device = torch.device("cuda")
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)


def get_dataloader(X_train : torch.tensor, X_val : torch.tensor, y_train : torch.tensor, y_val : torch.tensor, batch_size : int=20000, num_workers : int=8):
    train_dataloader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, num_workers=num_workers)
    val_dataloader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, num_workers=num_workers)
    return train_dataloader, val_dataloader

def load_vectors_to_np_arrays(encoder_dict : dict) -> np.ndarray:
    with open("tmp\\dataset_dumps\\preprocess_dumps\\embeddings\\glove_reddit_dict.pkl", "rb") as file:
        embeddings_dict = pickle.load( file)
    embeddings_dim = embeddings_dict["hello"].shape[0]
    embeddings = np.zeros((len(encoder_dict), embeddings_dim))
    for key, idx in encoder_dict.items():
        embeddings[idx] = embeddings_dict.get(key, np.zeros((embeddings_dim))).astype(np.float32)
    return torch.tensor(embeddings)

In [4]:
train_dataloader, val_dataloader = get_dataloader(X_train, X_val, y_train, y_val, batch_size=1000, num_workers=8)
embeddings = load_vectors_to_np_arrays(encoder_dict)

# nn.Module

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda")
from tqdm import tqdm

def set_seed(random_state : int):
    torch.manual_seed(random_state)
    torch.cuda.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    np.random.seed(random_state)
    random.seed(random_state)

class CNN(nn.Module):
    def __init__(self, pretrained_embeddings : torch.tensor=None, freeze_embeddings=False, emb_n_dims=300, vocab_size : int=None, token_seq_size=256, n_classes=1013):
        super(CNN, self).__init__()
        self.token_seq_size = token_seq_size
        self.conv_kernel_size = 10
        self.conv_out = 100
        self.unfreeze_epoch = 30
        self.n_classes = n_classes
        self.freeze_embeddings = freeze_embeddings
        if pretrained_embeddings is not None:
            self.vocab_size, self.emb_n_dims = pretrained_embeddings.shape
            self.embeddings_layer = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        else:
            self.vocab_size, self.emb_n_dims = vocab_size, emb_n_dims
            self.embeddings_layer = nn.Embedding(num_embeddings=vocab_size, \
                                                                  embedding_dim=self.emb_n_dims,padding_idx=0, max_norm=5.0)
            
        self.conv = nn.Conv1d(self.emb_n_dims,self.conv_out,kernel_size=self.conv_kernel_size,padding=4)
        self.conv_kernel_sizes= [5, 15, 30]
        self.convolutions = nn.ModuleList([nn.Conv1d(self.emb_n_dims,self.conv_out,kernel_size=kernel_size,padding=4) for kernel_size in self.conv_kernel_sizes])
        self.linear = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(self.conv_out * len(self.convolutions), self.conv_out),
            nn.ReLU(),
            nn.Linear(self.conv_out, self.n_classes)
        )
        
    def forward(self, input_ids):
        x = self.embeddings_layer(input_ids).float()
#         print("embeddings", x.shape, sep=' ')
#         print(x)
#         return None
        x = x.permute(0, 2, 1)
#         print("permute", x.shape, sep=' ')
#         x = self.conv(x)
        x = [conv1d(x) for conv1d in self.convolutions]
#         print("conv", x.shape, sep=' ')
        x = [F.relu(conv_res) for conv_res in x]
#         print("relu", x.shape, sep=' ')
        x = [F.max_pool1d(conv_res, conv_res.shape[2]) for conv_res in x]
#         print("pooling", (len(x), len(x[0])), sep=' ')
        x = torch.cat([conv_res.squeeze(dim=2) for conv_res in x], dim=1)
        x = self.linear(x)
#         print("linear", x.shape, sep=' ')
        x = F.log_softmax(x, dim=1)
        return x
    
    def fit(self, optimizer, scheduler, criterion, dataloader, n_iter : int):
        self = self.cuda()
        self.train()
        for epoch in range(n_iter):
            if not(self.freeze_embeddings) and epoch == self.unfreeze_epoch:
                self.embeddings_layer.weight.requires_grad = True
                print("EMBEDDINGS UNLEASHED ! ! !")
            for X, y in dataloader:
                X = X.view(-1, self.token_seq_size)
                X = X.cuda()
                y = y.cuda()
                optimizer.zero_grad()
                output = self(X)
                loss = criterion(output, y)
                loss.backward()
                optimizer.step()
                del X
                del y
            if epoch % 10 == 0:
                print(f"ep{epoch}: {loss}")
                path = f"tmp\\model_dumps\\cnn_text\\model_{self.conv_out}conv_{str(self.conv_kernel_sizes).replace(', ', '_')}kernelsize_{n_iter}ep_freezeemb_{self.freeze_embeddings}.pt"
                print(optimizer.param_groups[0]["lr"])
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                    }, path
                )
            scheduler.step()
            torch.cuda.empty_cache()
            


In [15]:
import torch.optim as optim
learning_rate = 5e-01
n_iter = 100
model = CNN(embeddings, freeze_embeddings=True, emb_n_dims=300, vocab_size=len(encoder_dict))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate, weight_decay=1e-05)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

In [16]:
model.fit(optimizer, scheduler, criterion, train_dataloader, n_iter)

ep0: 6.916538238525391
0.5
ep10: 2.6911909580230713
0.40853640344377334
ep20: 0.8724244236946106
0.3338039858775471
ep30: 0.48165076971054077
0.27274215969121846
ep40: 0.30965179204940796
0.22285020197547536
ep50: 0.22045207023620605
0.18208484004355835
ep60: 0.1865234673023224
0.1487765713460602
ep70: 0.08325999975204468
0.12156129074883078
ep80: 0.18253296613693237
0.09932442504102033
ep90: 0.09914611279964447
0.08115528676075824


In [17]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


def test(model, X, y): 
    X = X.cpu()
    model = model.cpu()
    model = model.eval()
    y_pred = torch.argmax(model(X), axis=1)
    metrics = {"accuracy": accuracy_score(y, y_pred),
               "f1_score": f1_score(y, y_pred, average="macro"),
               "precision_score": precision_score(y, y_pred, average="macro"),
               "recall_score": recall_score(y, y_pred, average="macro")}
#     print(y_pred, y_test)
    return metrics

test(model, X_test[:10000], y_test[:10000])

{'accuracy': 0.6039,
 'f1_score': 0.5937924449485792,
 'precision_score': 0.615643462199326,
 'recall_score': 0.6031703427817491}

In [44]:
import sys
sys.getsizeof(train_dataloader)
torch.cuda.empty_cache()

In [18]:
print(torch._C._cuda_getCompiledVersion(), 'cuda compiled version')

11010 cuda compiled version


In [74]:
torch.cuda.empty_cache()

In [6]:
import torch.optim as optim
set_seed(42)

learning_rate = 5e-01
n_iter = 100
model = CNN(embeddings, freeze_embeddings=False, emb_n_dims=300, vocab_size=len(encoder_dict))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate, weight_decay=1e-05)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
model.fit(optimizer, scheduler, criterion, train_dataloader, n_iter)

C:\Users\TOPAPEC\anaconda3\envs\coursework\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


ep0: 6.914767265319824
0.5
ep10: 3.0262460708618164
0.40853640344377334
ep20: 0.9993476867675781
0.3338039858775471
EMBEDDINGS UNLEASHED ! ! !
ep30: 0.37133246660232544
0.27274215969121846
ep40: 0.16381075978279114
0.22285020197547536
ep50: 0.1938459575176239
0.18208484004355835
ep60: 0.18892142176628113
0.1487765713460602
ep70: 0.06988176703453064
0.12156129074883078
ep80: 0.06524232029914856
0.09932442504102033
ep90: 0.09676981717348099
0.08115528676075824


In [7]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


def test(model, X, y): 
    X = X.cpu()
    model = model.cpu()
    model = model.eval()
    y_pred = torch.argmax(model(X), axis=1)
    metrics = {"accuracy": accuracy_score(y, y_pred),
               "f1_score": f1_score(y, y_pred, average="macro"),
               "precision_score": precision_score(y, y_pred, average="macro"),
               "recall_score": recall_score(y, y_pred, average="macro")}
#     print(y_pred, y_test)
    return metrics

test(model, X_test[:10000], y_test[:10000])

{'accuracy': 0.6311,
 'f1_score': 0.6190168009433548,
 'precision_score': 0.6397481581613659,
 'recall_score': 0.628543757167584}